In [1]:
from Bio import SeqIO # Biopython for fast parsing of FASTA data
import pandas as pd # pandas for dataframes to organize outputs

In [2]:
def complement(seq):
    """
    Takes in a sequence and output the complement of that sequence. Note, this 
    does not reverse the sequence so the output is not the reverse complement.

    seq: String. The sequence to get the complement of.
    Return: String. The complement of the input sequence.
    
    """
    comp = {'A':'T', 'T':'A', 'G':'C', 'C':'G'}
    comp_seq = "".join(comp.get(base, base) for base in seq)
    return comp_seq

In [3]:
# Load in the text file with the mutation targets
mutations = pd.read_csv('somatic_mutation_data.txt', sep='\t')
mutations

,chromosome,chromosome_start,chromosome_end,chromosome_strand,assembly_version,reference_genome_allele,mutated_to_allele
0,6,107956302,107956302,+,GRCh37,G,A
1,6,107956302,107956302,-,GRCh37,C,T
2,6,107956312,107956312,+,GRCh37,A,G
3,6,107956312,107956312,-,GRCh37,T,C
4,X,76920372,76920372,+,GRCh37,G,A
5,X,76920372,76920372,-,GRCh37,C,T
6,1,229781704,229781704,+,GRCh37,C,T
7,1,229781704,229781704,-,GRCh37,G,A
8,17,56738869,56738869,+,GRCh37,G,A
9,17,56738869,56738869,-,GRCh37,C,T


In [4]:
sequences = [] # Store the target sequences

for ind in range(len(mutations)):
    # Load information from mutation data table
    chr = 'chr' + mutations['chromosome'][ind] # Chromosome number of target
    loc = mutations['chromosome_start'][ind] - 1 # Location of target
    sense = mutations['chromosome_strand'][ind].strip() # Strand information
    ref = mutations['reference_genome_allele'][ind] # Expected reference base
    
    for sequence in SeqIO.parse(open('hg19.fa'), 'fasta'):
        if sequence.id == chr:
            seq = str(sequence.seq).upper()
            break
    
    # Check if the base at location is the same as the expected reference
    if seq[loc] != ref and sense == '+':
        raise Exception("Error in parsing sequence")

    # Get the 200bp sequence
    snp_seq = seq[loc-100:loc+100]

    # Check to see if the complement sequence needs to be calculated and append
    if sense == '+':
        sequences.append(snp_seq)
    else:
        sequences.append(complement(snp_seq))

In [5]:
# Get the counts of A, C, G, T in each extracted sequences
count_A = [sect.count('A') for sect in sequences]
count_C = [sect.count('C') for sect in sequences]
count_G = [sect.count('G') for sect in sequences]
count_T = [sect.count('T') for sect in sequences]

In [6]:
# Sanity check to make sure all the values add back up to 200 so that there wiil
#  not be any double or missing count
check_sum = [sum(x) for x in zip(count_A, count_C, count_G, count_T)]
for count, total in enumerate(check_sum):
    if total != 200:
        raise Exception(f"Mutation {count} individual single base total is not 
                        200")
check_sum

[200, 200, 200, 200, 200, 200, 200, 200, 200, 200]

#### Get the 2-Gram counts

In [7]:
count_AA = [sect.count('AA') for sect in sequences]
count_AC = [sect.count('AC') for sect in sequences]
count_AG = [sect.count('AG') for sect in sequences]
count_AT = [sect.count('AT') for sect in sequences]

In [8]:
count_CA = [sect.count('CA') for sect in sequences]
count_CC = [sect.count('CC') for sect in sequences]
count_CG = [sect.count('CG') for sect in sequences]
count_CT = [sect.count('CT') for sect in sequences]

In [9]:
count_GA = [sect.count('GA') for sect in sequences]
count_GC = [sect.count('GC') for sect in sequences]
count_GG = [sect.count('GG') for sect in sequences]
count_GT = [sect.count('GT') for sect in sequences]

In [10]:
count_TA = [sect.count('TA') for sect in sequences]
count_TC = [sect.count('TC') for sect in sequences]
count_TG = [sect.count('TG') for sect in sequences]
count_TT = [sect.count('TT') for sect in sequences]

#### Put all the results back together

In [11]:
mutations['Sequence'] = sequences
mutations['A count'] = count_A
mutations['C count'] = count_C
mutations['G count'] = count_G
mutations['T count'] = count_T
mutations['AA count'] = count_AA
mutations['AC count'] = count_AC
mutations['AG count'] = count_AG
mutations['AT count'] = count_AT
mutations['CA count'] = count_CA
mutations['CC count'] = count_CC
mutations['CG count'] = count_CG
mutations['CT count'] = count_CT
mutations['GA count'] = count_GA
mutations['GC count'] = count_GC
mutations['GG count'] = count_GG
mutations['GT count'] = count_GT
mutations['TA count'] = count_TA
mutations['TC count'] = count_TC
mutations['TG count'] = count_TG
mutations['TT count'] = count_TT

### Output

In [12]:
mutations

,chromosome,chromosome_start,chromosome_end,chromosome_strand,assembly_version,reference_genome_allele,mutated_to_allele,Sequence,A count,C count,...,CG count,CT count,GA count,GC count,GG count,GT count,TA count,TC count,TG count,TT count
0,6,107956302,107956302,+,GRCh37,G,A,GGCCTGCAACGTCATCGTGAACGGCACGCGCGGCGCCGCCGCCGAG...,35,74,...,29,11,14,33,13,7,3,7,10,21
1,6,107956302,107956302,-,GRCh37,C,T,CCGGACGTTGCAGTAGCACTTGCCGTGCGCGCCGCGGCGGCGGCTC...,21,70,...,33,14,11,29,22,8,2,15,8,35
2,6,107956312,107956312,+,GRCh37,A,G,GTCATCGTGAACGGCACGCGCGGCGCCGCCGCCGAGGGCGCTAAGA...,36,73,...,29,10,15,33,14,7,3,7,9,20
3,6,107956312,107956312,-,GRCh37,T,C,CAGTAGCACTTGCCGTGCGCGCCGCGGCGGCGGCTCCCGCGATTCT...,20,71,...,33,15,10,29,22,8,2,16,7,36
4,X,76920372,76920372,+,GRCh37,G,A,AGAGTAAAAAACAATAAAAAAAGAACTATATATCTGGGGGTGAAAT...,97,25,...,0,10,7,3,2,6,24,4,8,58
5,X,76920372,76920372,-,GRCh37,C,T,TCTCATTTTTTGTTATTTTTTTCTTGATATATAGACCCCCACTTTA...,58,20,...,3,7,10,0,4,11,21,8,14,97
6,1,229781704,229781704,+,GRCh37,C,T,AGGCTGTTGTGTCAGCTGTTACACTGCTGAGGCTGCTACTGAACTG...,38,56,...,2,24,11,14,7,19,7,15,23,54
7,1,229781704,229781704,-,GRCh37,G,A,TCCGACAACACAGTCGACAATGTGACGACTCCGACGATGACTTGAC...,54,52,...,14,11,24,2,12,11,2,19,9,38
8,17,56738869,56738869,+,GRCh37,G,A,GAACAAGTCAAGCTGCGATAACATTTTGGCAACCTTGTCAAGGAGG...,42,53,...,14,16,14,16,16,9,2,17,11,46
9,17,56738869,56738869,-,GRCh37,C,T,CTTGTTCAGTTCGACGCTATTGTAAAACCGTTGGAACAGTTCCTCC...,46,59,...,16,14,16,14,9,13,5,13,10,42
